In [35]:
%load_ext autoreload
%autoreload 2

import os, sys
import copy
import socket
from tqdm import tqdm
import torch
import pickle
from torch import optim
from torch.utils.tensorboard import SummaryWriter

#!pip install networkx matplotlib
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
from libs.topology_manager import *
from libs import fl, nn, agg, data, log

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
# Save Logs To File (info | debug | warning | error | critical) [optional]
log.init("info")
#log.init("info", "federated.log")
#log.init("debug", "flkafka.log")

In [37]:
adj_mat = np.array([[0,1,1,1,1,1],
             [1,0,0,0,0,0],
             [1,0,0,0,0,0],
             [1,0,0,0,0,0],
             [1,0,0,0,0,0], 
             [1,0,0,0,0,0]])

node_types = {'aggregator': [0,1], 'trainer': [2,3], 'broadcaster': [4,5]}

di_graph = nx.DiGraph(adj_mat)
#nx.draw(di_graph)

In [38]:
class FedArgs():
    def __init__(self):
        self.num_clients = len(adj_mat)
        self.epochs = 10
        self.local_rounds = 1
        self.client_batch_size = 32
        self.test_batch_size = 128
        self.learning_rate = 1e-4
        self.weight_decay = 1e-5
        self.cuda = False
        self.seed = 1
        self.topic = 'pyflx'
        self.tb = SummaryWriter('../out/runs/flkafka', comment="Mnist Distributed Federated training")

fedargs = FedArgs()

In [39]:
use_cuda = fedargs.cuda and torch.cuda.is_available()
torch.manual_seed(fedargs.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

In [40]:
host = socket.gethostname()
clients = [host + "(" + str(client + 1) + ")" for client in range(fedargs.num_clients)]
ctp = DistributedTopology(adj_mat, clients, node_types)

In [41]:
#Initialize Global and Client models
global_model = nn.ModelMNIST()
client_models = {client: copy.deepcopy(global_model) for client in clients}

# Function for training
def train_model(_model, train_loader, fedargs, device):
    model, loss = fl.client_update(_model,
                                train_loader,
                                fedargs.learning_rate,
                                fedargs.weight_decay,
                                fedargs.local_rounds,
                                device)
    model_update = agg.sub_model(_model, model)
    return model_update, model, loss

In [42]:
# Load MNIST Data to clients
train_data, test_data = data.load_dataset("mnist")
clients_data = data.split_data(train_data, clients)
client_train_loaders, client_test_loaders = data.load_client_data(clients_data, fedargs.client_batch_size, 0.2, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=fedargs.test_batch_size, shuffle=True, **kwargs)

clients_info = {
        client: {"train_loader": client_train_loaders[client],
                 "test_loader": client_test_loaders[client]}
        for client in clients
    }

In [43]:
import asyncio
import time

def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)

    return wrapped

@background
def process(client, epoch, ctp, model, train_loader, test_loader, fedargs, device):
    log.info("Processing Client {}".format(client))
    
    # Consume and Average, epoch passed is actually prev epoch, for which we want to consume updates
    client_model_updates = ctp.consume_model(client, fedargs.topic, model, epoch - 1)
    # Pop one's own update
    if client in client_model_updates:
        client_model_updates.pop(client)
    log.info("Client {} received {} model update(s) from {}".format(client, 
                                                                    len(client_model_updates), 
                                                                    list(client_model_updates.keys())))
    if len(client_model_updates) != 0:
        model = fl.federated_avg(client_model_updates, model)

    # Train    
    model_update, model, loss = train_model(model, train_loader, fedargs, device)
    
    # Publish
    ctp.produce_model(client, fedargs.topic, model_update, epoch)

    # Plot and Log
    for local_epoch, loss in enumerate(list(loss.values())):
        fedargs.tb.add_scalars("Training Loss/" + client, {str(epoch): loss}, str(local_epoch + 1))

    log.jsondebug(loss, "Epoch {} of {} : Federated Training loss, Client {}".format(epoch, fedargs.epochs, client))
    log.modeldebug(model, "Epoch {} of {} : Client {} Update".format(epoch, fedargs.epochs, client))

    # Test
    test_output = fl.eval(model, test_loader, device)
    fedargs.tb.add_scalar("Accuracy/" + client, test_output["accuracy"], epoch)
    log.jsoninfo(test_output, "Test Outut after Epoch {} of {} for Client {}".format(epoch, fedargs.epochs, client))
    
    return model

In [44]:
import time
start_time = time.time()

# Federated Training
for _epoch in tqdm(range(fedargs.epochs)):

    epoch = _epoch + 1
    log.info("Federated Training Epoch {} of {}".format(epoch, fedargs.epochs))

    # Clients  
    loop = asyncio.get_event_loop()
    tasks = [process(client, epoch, ctp, client_models[client],
                     clients_info[client]['train_loader'],
                     clients_info[client]['test_loader'],
                     fedargs, device) for client in clients]
    models = loop.run_until_complete(asyncio.gather(*tasks))
    client_models = {client: model for client, model in zip(clients, models)}
    
print(time.time() - start_time)

  0%|          | 0/10 [00:00<?, ?it/s]2021-08-18 14:30:39,533 - <ipython-input-44-a4d02bc96df0>::<module>(l:8) : Federated Training Epoch 1 of 10 [MainProcess : MainThread (INFO)]
2021-08-18 14:30:39,616 - <ipython-input-43-3d420fa12194>::process(l:12) : Processing Client bladecluster.iitp.org(1) [MainProcess : asyncio_2 (INFO)]
2021-08-18 14:30:39,617 - <ipython-input-43-3d420fa12194>::process(l:12) : Processing Client bladecluster.iitp.org(4) [MainProcess : asyncio_1 (INFO)]
2021-08-18 14:30:39,654 - <ipython-input-43-3d420fa12194>::process(l:12) : Processing Client bladecluster.iitp.org(5) [MainProcess : asyncio_3 (INFO)]
2021-08-18 14:30:39,675 - <ipython-input-43-3d420fa12194>::process(l:12) : Processing Client bladecluster.iitp.org(3) [MainProcess : asyncio_0 (INFO)]
2021-08-18 14:30:39,676 - <ipython-input-43-3d420fa12194>::process(l:12) : Processing Client bladecluster.iitp.org(2) [MainProcess : asyncio_4 (INFO)]
2021-08-18 14:30:39,680 - <ipython-input-43-3d420fa12194>::proces

2021-08-18 14:31:23,656 - <ipython-input-43-3d420fa12194>::process(l:19) : Client bladecluster.iitp.org(5) received 5 model update(s) from ['bladecluster.iitp.org(1)', 'bladecluster.iitp.org(4)', 'bladecluster.iitp.org(3)', 'bladecluster.iitp.org(2)', 'bladecluster.iitp.org(6)'] [MainProcess : asyncio_3 (INFO)]
2021-08-18 14:31:23,662 - <ipython-input-43-3d420fa12194>::process(l:19) : Client bladecluster.iitp.org(3) received 5 model update(s) from ['bladecluster.iitp.org(1)', 'bladecluster.iitp.org(4)', 'bladecluster.iitp.org(5)', 'bladecluster.iitp.org(2)', 'bladecluster.iitp.org(6)'] [MainProcess : asyncio_0 (INFO)]
2021-08-18 14:31:23,665 - <ipython-input-43-3d420fa12194>::process(l:19) : Client bladecluster.iitp.org(2) received 5 model update(s) from ['bladecluster.iitp.org(1)', 'bladecluster.iitp.org(4)', 'bladecluster.iitp.org(3)', 'bladecluster.iitp.org(5)', 'bladecluster.iitp.org(6)'] [MainProcess : asyncio_1 (INFO)]
2021-08-18 14:31:23,677 - <ipython-input-43-3d420fa12194>::pr

2021-08-18 14:31:58,208 - <ipython-input-43-3d420fa12194>::process(l:19) : Client bladecluster.iitp.org(4) received 5 model update(s) from ['bladecluster.iitp.org(1)', 'bladecluster.iitp.org(6)', 'bladecluster.iitp.org(3)', 'bladecluster.iitp.org(5)', 'bladecluster.iitp.org(2)'] [MainProcess : asyncio_1 (INFO)]
2021-08-18 14:31:58,306 - <ipython-input-43-3d420fa12194>::process(l:19) : Client bladecluster.iitp.org(3) received 5 model update(s) from ['bladecluster.iitp.org(1)', 'bladecluster.iitp.org(6)', 'bladecluster.iitp.org(5)', 'bladecluster.iitp.org(2)', 'bladecluster.iitp.org(4)'] [MainProcess : asyncio_3 (INFO)]
2021-08-18 14:31:58,335 - <ipython-input-43-3d420fa12194>::process(l:19) : Client bladecluster.iitp.org(5) received 5 model update(s) from ['bladecluster.iitp.org(1)', 'bladecluster.iitp.org(6)', 'bladecluster.iitp.org(3)', 'bladecluster.iitp.org(2)', 'bladecluster.iitp.org(4)'] [MainProcess : asyncio_2 (INFO)]
2021-08-18 14:31:58,339 - <ipython-input-43-3d420fa12194>::pr

2021-08-18 14:32:33,414 - <ipython-input-43-3d420fa12194>::process(l:19) : Client bladecluster.iitp.org(5) received 5 model update(s) from ['bladecluster.iitp.org(2)', 'bladecluster.iitp.org(4)', 'bladecluster.iitp.org(1)', 'bladecluster.iitp.org(3)', 'bladecluster.iitp.org(6)'] [MainProcess : asyncio_3 (INFO)]
2021-08-18 14:32:33,427 - <ipython-input-43-3d420fa12194>::process(l:19) : Client bladecluster.iitp.org(3) received 5 model update(s) from ['bladecluster.iitp.org(5)', 'bladecluster.iitp.org(2)', 'bladecluster.iitp.org(4)', 'bladecluster.iitp.org(1)', 'bladecluster.iitp.org(6)'] [MainProcess : asyncio_1 (INFO)]
2021-08-18 14:32:33,429 - <ipython-input-43-3d420fa12194>::process(l:19) : Client bladecluster.iitp.org(6) received 5 model update(s) from ['bladecluster.iitp.org(5)', 'bladecluster.iitp.org(2)', 'bladecluster.iitp.org(4)', 'bladecluster.iitp.org(1)', 'bladecluster.iitp.org(3)'] [MainProcess : asyncio_4 (INFO)]
2021-08-18 14:32:33,430 - <ipython-input-43-3d420fa12194>::pr

2021-08-18 14:33:09,345 - <ipython-input-43-3d420fa12194>::process(l:19) : Client bladecluster.iitp.org(6) received 5 model update(s) from ['bladecluster.iitp.org(1)', 'bladecluster.iitp.org(2)', 'bladecluster.iitp.org(4)', 'bladecluster.iitp.org(3)', 'bladecluster.iitp.org(5)'] [MainProcess : asyncio_4 (INFO)]
2021-08-18 14:33:09,373 - <ipython-input-43-3d420fa12194>::process(l:19) : Client bladecluster.iitp.org(4) received 5 model update(s) from ['bladecluster.iitp.org(1)', 'bladecluster.iitp.org(2)', 'bladecluster.iitp.org(3)', 'bladecluster.iitp.org(5)', 'bladecluster.iitp.org(6)'] [MainProcess : asyncio_1 (INFO)]
2021-08-18 14:33:09,375 - <ipython-input-43-3d420fa12194>::process(l:19) : Client bladecluster.iitp.org(3) received 5 model update(s) from ['bladecluster.iitp.org(1)', 'bladecluster.iitp.org(2)', 'bladecluster.iitp.org(4)', 'bladecluster.iitp.org(5)', 'bladecluster.iitp.org(6)'] [MainProcess : asyncio_5 (INFO)]
2021-08-18 14:33:27,172 - /home/harsh_1921cs01/hub/F3IA/fl/li

...

2021-08-18 14:34:20,762 - <ipython-input-43-3d420fa12194>::process(l:19) : Client bladecluster.iitp.org(5) received 5 model update(s) from ['bladecluster.iitp.org(2)', 'bladecluster.iitp.org(1)', 'bladecluster.iitp.org(3)', 'bladecluster.iitp.org(4)', 'bladecluster.iitp.org(6)'] [MainProcess : asyncio_0 (INFO)]
2021-08-18 14:34:37,323 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx. ^C to exit. [MainProcess : asyncio_2 (INFO)]
2021-08-18 14:34:38,101 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx. ^C to exit. [MainProcess : asyncio_5 (INFO)]
2021-08-18 14:34:38,350 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx. ^C to exit. [MainProcess : asyncio_1 (INFO)]
2021-08-18 14:34:38,448 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to t

2021-08-18 14:35:12,086 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx. ^C to exit. [MainProcess : asyncio_4 (INFO)]
2021-08-18 14:35:12,287 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : asyncio_4 (INFO)]
2021-08-18 14:35:12,806 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx. ^C to exit. [MainProcess : asyncio_3 (INFO)]
2021-08-18 14:35:13,101 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : asyncio_3 (INFO)]
2021-08-18 14:35:13,549 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx. ^C to exit. [MainProcess : asyncio_5 (INFO)]
2021-08-18 14:35:13,550 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::delivery_report(l:50) : User record b'bladeclu

2021-08-18 14:35:48,028 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx. ^C to exit. [MainProcess : asyncio_2 (INFO)]
2021-08-18 14:35:48,236 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : asyncio_2 (INFO)]
2021-08-18 14:35:48,663 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::delivery_report(l:50) : User record b'bladecluster.iitp.org(1)' successfully produced to pyflx [0] at offset 216 [MainProcess : asyncio_4 (INFO)]
2021-08-18 14:35:48,669 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx. ^C to exit. [MainProcess : asyncio_3 (INFO)]
2021-08-18 14:35:49,013 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx. ^C to exit. [MainProcess : asyncio_0 (INFO)]
2021-08-18 14:35:49,184 - /home/harsh_1921cs01/hub/F3IA

2021-08-18 14:36:23,907 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx. ^C to exit. [MainProcess : asyncio_3 (INFO)]
2021-08-18 14:36:23,912 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::delivery_report(l:50) : User record b'bladecluster.iitp.org(2)' successfully produced to pyflx [0] at offset 222 [MainProcess : asyncio_2 (INFO)]
2021-08-18 14:36:24,137 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : asyncio_3 (INFO)]
2021-08-18 14:36:24,588 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::delivery_report(l:50) : User record b'bladecluster.iitp.org(1)' successfully produced to pyflx [0] at offset 223 [MainProcess : asyncio_4 (INFO)]
2021-08-18 14:36:24,590 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx. ^C to exit. [MainProcess : asyncio_5 (INFO)]
2021-08-1

350.2824819087982
